<h2>Embedding Projector in TensorBoard</h2>
<p>We can represent the high dimensional embeddings graphicall using <b>TensorBoard Embedding projector</b>. An Embeddding in TensorFlow is mapping like the word to vector of real numbers. This projector is helpful in visualizing, examining and understanding embedding layers. Here we will learn visualize the training layer as shown below.</p>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding.jpg?raw=\" width="850" height="450">
<p>We will be using TensorBoard to visualize the embedding layer generated for classifying moview review data.</p>

In [1]:
# Adding the TensorBoard extension to the notebook
%load_ext tensorboard

In [2]:
# Importing required libraries
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector

<h3>IMDB dataset of TensorFlow</h3>
<p>This dataset contains 25,000 movies reviews from IMDB labelled either positive or negative. Reviews have been preprocessed and each review is encoded as a sequence of word indexes or integers. <br>
For our convenience, words in dataset are indexed by overall frequency in the dataset, so that for instance the integer 3 encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: only consider top 10,000 most common words but discard the top 20 most common words. The integer 0 does not stand for a specific word, but it is used to encode any unknown word. Later we will be removing this row from the visualization.

In [3]:
# Loading the IMDB dataset from tensorflow_dataset
(training_data, tests_data), info = tfds.load(
    "imdb_reviews/subwords8k",
    split=(tfds.Split.TRAIN, tfds.Split.TEST),
    with_info=True,
    as_supervised=True,
)
#Encoding the text 
encoder = info.features["text"].encoder

# shuffle and pad the training and test data.
#This process will take 2-3 minutes based on your internet speed.
training_batches = training_data.shuffle(1000).padded_batch(
    10, padded_shapes=((None,), ())
)
tests_batches = tests_data.shuffle(1000).padded_batch(
    10, padded_shapes=((None,), ())
)
training_batch, training_labels = next(iter(training_batches))

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteO5ASEY/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteO5ASEY/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteO5ASEY/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


<h3>Keras Embedding Layer</h3>
<p>A keras embedding layer can be used to train an embedding for each word in the vocabulary. Each word or subword will be associated with 16-dimensional vector or embedding that will be trained by the model.

In [4]:
# Create an embedding layer with 16 dimensions
embedding_dim = 16
embedding = tf.keras.layers.Embedding(encoder.vocab_size, embedding_dim)

# Train the model with the embeddling layer defined above
model = tf.keras.Sequential(
    [
        embedding, # The embedding layer should be the first layer in a model.
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1),
    ]
)

# Compile the  model with adam optimizer
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Train model with 5 epochs
history = model.fit(
    training_batches, epochs=5, validation_data=tests_batches, validation_steps=20
)

Epoch 1/5
2500/2500 [==============================] - 10s 4ms/step - loss: 0.6190 - accuracy: 0.5756 - val_loss: 0.3973 - val_accuracy: 0.8800
Epoch 2/5
2500/2500 [==============================] - 9s 3ms/step - loss: 0.3071 - accuracy: 0.8729 - val_loss: 0.3638 - val_accuracy: 0.8500
Epoch 3/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.2427 - accuracy: 0.9042 - val_loss: 0.2790 - val_accuracy: 0.8950
Epoch 4/5
2500/2500 [==============================] - 9s 3ms/step - loss: 0.2060 - accuracy: 0.9210 - val_loss: 0.3336 - val_accuracy: 0.8550
Epoch 5/5
2500/2500 [==============================] - 9s 4ms/step - loss: 0.1822 - accuracy: 0.9301 - val_loss: 0.4197 - val_accuracy: 0.8550


<h3>Saving the log data for TensorBoard</h3>
<p>TensorBoard will read the tensors and metadata of the TensorFlow project from the specified <b>log_dir</b> directory. Here we will be using the directory <b>/logs/imdb-example/</b><br>
For visualization, we will be saving checkpoint to the specified directory along with metadata to understand which layer to visualize.

In [5]:
# Set up a logs directory, so Tensorboard knows where to look for files
# If exist the logs/imdb-example it will not recreate it.
log_dir='/logs/imdb-example/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save labels of dataset separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for subwords in encoder.subwords:
    f.write("{}\n".format(subwords))

  # Fill in the rest of the labels with "unknown"
  for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
    f.write("unknown #{}\n".format(unknown))


# Save the weights we want to analyse as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, so
# we will remove that value.
weights = tf.Variable(model.layers[0].get_weights()[0][1:])

# Create a checkpoint from embedding, the filename and key are
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()

# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [6]:
# Open TensorBoard
%tensorboard --logdir /logs/imdb-example/

<IPython.core.display.Javascript object>

<h3>Analysis</h3>
<p>The TensorBoard projector is a great tool for analyzing our data and seeing embedding values relative to each other. The dashboard allows for searching the specific terms and highlights words which are nearby in the emmbedding space. <br>
From this example, we can see that Wes Anderson and Alfred Hitchcock are both neutral terms, but they are referenced in different contexts.
<img src="https://www.tensorflow.org/tensorboard/images/embedding_projector_hitchcock.png?raw=1" width="850" height="450">
Hithcock is closer associated to words like nightmare, which likely relates to his work in horror movies. While Anderson is closer to the word heart, reflecting his heartwarming style.
<img src="https://www.tensorflow.org/tensorboard/images/embedding_projector_anderson.png?raw=1" width="850" height="450">